<a href="https://colab.research.google.com/github/peterhaasme/nc_covid19_dashboard/blob/master/nc_covid19_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import Libraries
import pandas as pd
pd.set_option('display.max_rows', None)
from datetime import datetime, timedelta 
import plotly.express as px
import plotly.graph_objects as go